In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '..')
import fashion.preprocessing as prep
# display all columns
pd.set_option('display.max_columns', None)

# show all results of the notebook not just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
shops = prep.load_shops('../data/20200120_filiali.csv')
shops.head()

In [ ]:
import importlib
importlib.reload(prep)

sales17 = prep.load_sales('../data/20200120_sales17.csv', shops)
sales17.head()
len(sales17)

In [ ]:
sales17.dtypes

In [ ]:
importlib.reload(prep)

prods = prep.load_products('../data/20200120_barcode.csv')
prods.head()

In [ ]:
which = 'ProductID'
len(prods)
prods[which].value_counts().head(20)
prods[which].unique()
prods[(prods[which] == 67527) ]#.ColorDescription.unique()

In [ ]:

def explore(df):
    
    # look at values for each column
    for col in df.columns:
        
        print(col)
        uniques = df[col].unique()
        print(len(uniques))
        if len(uniques) > 100:
            continue
        counts = {u:np.sum(df.eval('{0}=="{1}"'.format(col, u))) for u in uniques}
        
        fig, ax = plt.subplots(figsize=(10,10))
        ax.bar(x=range(len(uniques)), height=[counts[u] for u in uniques], tick_label=uniques)
        ax.set_title(col)
        plt.show()
        

explore(barcode)
        


In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
_ = ax.hist(sales17.Net_Income, bins=100)
ax.set_yscale('log')
plt.show()

In [ ]:
sales1819 = prep.load_sales('../data/20200120_sales1819.csv', shops)
sales1819.head()
len(sales1819)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
_ = ax.hist(sales1819.Net_Income, bins=100)#, range=(-1000, 1000))
ax.set_yscale('log')
plt.show()

In [ ]:
#sales1819[abs(sales1819.Net_Income) > 100]
sales1819[sales1819.Volume > 10]

In [ ]:
len(shops)

In [ ]:
sales1819.head(10)
year = sales1819.Date

In [ ]:
test = pd.to_datetime(year.astype(str))
pd.DatetimeIndex(test).year.unique()

In [ ]:
minisales = sales17#.head(100000)
idx = pd.DatetimeIndex(minisales.Date.astype(str))
years = idx.year
months = idx.month
days = idx.day

#minisales18 = minisales[pd.DatetimeIndex(minisales.Date.astype(str)).year == 2018]
#years = pd.DatetimeIndex(minisales18.Date.astype(str)).year

years.value_counts()

In [ ]:
#months[years==2016].value_counts()
#months[years==2017].value_counts()
#months[years==2018].value_counts()
#months[years==2019].value_counts()

vcounts = months[years==2016].value_counts()
vcounts

sales = [vcounts.loc[m] if m in vcounts else 0 for m in xs]
sales

fig, ax = plt.subplots()
xs = range(1, 13)
for y in [2016, 2017, 2018, 2019]:
    vcounts = months[years==y].value_counts()
    ax.plot(xs, [vcounts.loc[m] if m in vcounts else 0 for m in xs]/np.sum(vcounts), label='{} ({})'.format(y, np.sum(vcounts)))
ax.legend()
ax.set_xlabel('month of the year')
ax.set_ylabel('fraction of yearly sales')
ax.set_title('sales17 data')
plt.show()



In [ ]:
minisales.head()
minisales.dtypes
minisales.memory_usage() / 10**9
np.sum(minisales.memory_usage() / 10**9)